## Kaggle 대회 정리

### 우주왕복선 타이타닉 : 어떤 탑승객이 다른차원으로 전송되었는지 예측하라!

* 대회명 : Spaceship Titanic : Predict which passengers are transported to an alternate dimension
* 링크 : https://www.kaggle.com/competitions/spaceship-titanic

**<목차>**
1. 대회이해하기
2. EDA\#01 : 데이터이해하기

In [17]:
# 라이브러리 로드

# 시스템 라이브러리
import os
import warnings
warnings.filterwarnings('ignore')

# 데이터 핸들링 라이브러리
import pandas as pd

### STEP01 : 데이터 로드 및 체크

In [11]:
# STEP[01-01] : 데이터설명과 맞는 파일이 다운로드되었는지 체크
# test.csv', 'train.csv', 'sample_submission.csv' 3개의 파일이 존재함을 확인 >>결론 PASS

os.listdir("./raw")

['.DS_Store', 'test.csv', 'train.csv', 'sample_submission.csv']

In [12]:
# STEP[01-02] : 각 데이터가 올바른 행수와 형태를 갖었는지 확인


## 경로설정
path_train = "./raw/train.csv"
path_test = "./raw/test.csv"


## pandas DataFrame으로 로드
raw_train = pd.read_csv(path_train)
raw_test = pd.read_csv(path_test)

In [14]:
# 유니크한 key인 PassengerId가 8693(kaggle홈페이지와 일치함)칼럼도 모두 있음. 결론 PASS
## [체크해야할 point] : Null 혹은 Nan 값이 존재하는것을 확인, 따라서 이 값을 처리할 계획을 염두해야함.


## info() 매소드를 통해 데이터 스키마 확인
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [15]:
# 유니크한 key인 PassengerId가 4277(kaggle홈페이지와 일치함)칼럼도 모두 있음. 결론 PASS
## [체크해야할 point] : 잘 보면 test데이터이기 때문에 train.csv에는 있는 타겟변수인 Transported 칼럼이 없음.

## info() 매소드를 통해 데이터 스키마 확인
raw_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


### STEP02 : 데이터 설명과 함께 육안으로 살펴보며 이해하기

값의 구성에 대해서 살펴보며 대략적인 이해를 진행함<br>
<br>
ex: PassengerId는 `gggg_pp`로 되어있다고 했는데, 실제 데이터보면서 확인<br>
ex: CryoSleep이 가사상태로 항해하는지 여부라고 되어있는데, 실제값을 통해 불리언값임을 확인 등<br>

**<train.csv 칼럼 스키마>**
* PassengerId : 승객에 대한 유니크한 id로 각 id는 `gggg_pp` 포멧으로 표현됩니다. 이 때, `gggg`는 그룹을 뜻하며 이들은 같이 여행하는 승객들을 의미합다. `pp`는 그룹내의 각 승객의 번호를 의미합니다.(그룹에 속하는 구성원들은 대개 가족인 경우가 흔하지만, 그렇다고 반드시 가족을 의미하는것은 아닙니다.)<br>
* HomePlanet : 승객들이 출발한 행성입니다. 일반적으로 승객들이 일생을 보낸 행성입니다.<br>
* CryoSleep - 가사(suspended animation)방식으로 항해하는것을 선택하였는지 여부입니다.(승객의 객실(cabin)을 통해 더 정확하게 구분할 수 있습니다.)<br>
* Cabin : 탑승객이 머무는 객실번호로 `deck/num/side` 포멧을 갖습니다. side의 경우 포트(port)를 의미하는 `P`나 배의 우현(Starboard)을 의미하는 `S`를 의미합니다.<br>
* Destination : 승객들이 내리고자하는 행성입니다. <br>
* Age : 탑승객의 나이 입니다. <br>
* VIP : 탑승객이 항해도중 VIP 서비스를 신청하고 지불하였는지 아닌지 여부 입니다.<br>
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck : 승객이 얼마나 왕복선의 럭셔리 편의시설 지불했는지 입니다.<br>
* Name : 승객의 The first, last names(이름)<br>
* Transported : 다른차원으로 전송되었는지 아닌지 여부를 나타냅니다. 이 칼럼이 여러분이 예측을 해야할 타겟변수입니다.<br>

In [16]:
raw_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


### 내가 찾은 결론
1. 데이터 이해 : 데이터를 설명과 함께 실제 육안으로 살펴보며 형태와 이후 처리과정에 대해서 대랴적으로 이해함 (연속형, 범주형 등)
2. 계획 : 
    * EDA 전략 : 연속형과 범주형이 존재하니 각 변수 타입별로 Transported와 비교해보자
    * EDA 가설 및 파생변수 계획 : 상상의 나래를 펼치는것이 중요하다.
        - PassengerId에서 특정그룹이나, 그룹내 순서가 영향이 있지 않을까? 
        - Cabin 변수의 특정 패턴이 있지 않을까? 그 패턴이 영향이 있지 않을까? 
        - Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDec 등 연속형 변수들을 구간화 하거나, 형태를 변경할 경우 패턴이 발생할것 같음
        - RoomService,FoodCourt,ShoppingMall,Spa,VRDec를 별도 칼럼형태에서 간소화할 필요가 있을듯
        - Name : 이름의 경우 성이라는 공통분모가 있기 때문에 가족구성원의 관계성이 영향을 주지 않을까? 예: 부모님/자식/형제/친척관계 등등